<a href="https://colab.research.google.com/github/staerkjoe/QuantRisk_Colab/blob/main/TicketCancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/classification-of-travel-purpose")

print("Path to dataset files:", path)

100%|██████████| 7.30M/7.30M [00:00<00:00, 40.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pkdarabi/classification-of-travel-purpose/versions/1


Link to dataset:
https://www.kaggle.com/datasets/pkdarabi/classification-of-travel-purpose

In [2]:
import tensorflow as tf
import tf_keras as keras
import pandas as pd
# note: tensorflow 2.17 onwards uses tf.keras 3.0 internally,
# which seem to be still bugged and inconsistent.
# We therefore use tf_keras in according to tensorflow version.

import numpy as np
import os
import time

In [4]:
for file in os.listdir(path):
    print(file)

train_data.csv


In [5]:
df = pd.read_csv(os.path.join(path, "train_data.csv"))

In [13]:
# drop columns that are unnecessary/not helpful for prediction

# List of columns to drop
drop_cols = [
    'BillID',               # Unique identifier
    'TicketID',             # Unique identifier
    'UserID',               # Sparse and semi-unique
    'BuyerMobile',          # Personal identifier
    'NationalCode',         # Personal identifier
    'HashPassportNumber_p', # Sparse and hashed
    'HashEmail',            # Sparse and hashed
    'CancelTime'            # Leaks target label
]

# Drop columns from DataFrame
df = df.drop(columns=drop_cols)

# Optional: Confirm result
print("Remaining columns:", df.columns.tolist())


Remaining columns: ['Created', 'DepartureTime', 'ReserveStatus', 'Male', 'Price', 'CouponDiscount', 'From', 'To', 'Domestic', 'VehicleType', 'VehicleClass', 'TripReason', 'Vehicle', 'Cancel']


In [26]:
# Columns Transformations

# Date FOrmat
df[['Created', 'DepartureTime']] = df[['Created', 'DepartureTime']].apply(pd.to_datetime)

# Price to €
exchange_rate = 0.00002027  # 1 IRR = 0.00002027 EUR
df['Price'] = (df['Price'] * exchange_rate).round(2)
df['CouponDiscount'] = (df['CouponDiscount'] * exchange_rate).round(2)


In [27]:
# Fearure Engineering
df['LeadTime'] = df['DepartureTime'] - df['Created']
df['IsWeekend'] = df['DepartureTime'].dt.weekday >= 5
df['DiscountRatio'] = df['CouponDiscount'] / df['Price']

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101017 entries, 0 to 101016
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   Created         101017 non-null  datetime64[ns] 
 1   DepartureTime   101017 non-null  datetime64[ns] 
 2   ReserveStatus   101017 non-null  int64          
 3   Male            101017 non-null  bool           
 4   Price           101017 non-null  float64        
 5   CouponDiscount  101017 non-null  float64        
 6   From            101017 non-null  object         
 7   To              101017 non-null  object         
 8   Domestic        101017 non-null  int64          
 9   VehicleType     93473 non-null   object         
 10  VehicleClass    62567 non-null   object         
 11  TripReason      101017 non-null  object         
 12  Vehicle         101017 non-null  object         
 13  Cancel          101017 non-null  int64          
 14  LeadTime        1010

In [29]:
df.head()

,Created,DepartureTime,ReserveStatus,Male,Price,CouponDiscount,From,To,Domestic,VehicleType,VehicleClass,TripReason,Vehicle,Cancel,LeadTime,IsWeekend,DiscountRatio
0,2022-10-23 09:38:49.110,2022-11-02 23:59:00,5,True,0.0,0.7,گرگان,تهران,1,NaN,False,Work,Plane,0,10 days 14:20:10.890000,False,inf
1,2022-08-15 14:51:43.160,2022-08-18 04:15:00,5,True,0.0,0.0,مشهد,تهران,1,NaN,False,Int,Plane,0,2 days 13:23:16.840000,False,NaN
2,2022-09-20 17:25:27.250,2022-09-21 11:00:00,3,False,0.0,0.0,شیراز,اهواز,1,VIP 2+1,True,Work,Bus,0,0 days 17:34:32.750000,False,NaN
3,2022-06-25 11:32:53.980,2022-06-26 08:30:00,2,False,0.0,0.0,تبریز,جلفا,1,3 ستاره اتوبوسي,NaN,Int,Train,0,0 days 20:57:06.020000,True,NaN
4,2022-06-01 11:30:53.633,2022-06-02 23:00:00,3,True,0.0,0.0,تهران,نجف آباد (اصفهان ),1,اسکانیا تک صندلی ۳۱نفره,True,Int,Bus,0,1 days 11:29:06.367000,False,NaN


In [23]:
df['Vehicle'].value_counts()

,count
Vehicle,
Bus,48960
Train,38450
Plane,12812
InternationalPlane,795


In [30]:
df['Price'].mean()

np.float64(0.0003952800023758378)